In [1]:
import requests
from bs4 import BeautifulSoup
import time

In [12]:
def parse_object_page(url, object_n, object_list):
    """
    Fonction pour parser une page contenant des informations sur un type d'objet.
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Safari/537.3'}
    
    response = requests.get(url, headers=headers)
    if response.status_code == 200 :

        soup = BeautifulSoup(response.text, "html.parser")
        table = soup.find('table', class_='ak-table')
        if table:
            for row in table.find_all('tr'):
                cells = row.find_all('td')
                if cells :
                    nom_index = 1
    
                    if object_n in ["weapons", "equipment"]:
                        type_index = 3
                        niveau_index = 4

                    else:
                        type_index = 2
                        niveau_index = 3

                    if object_n in ["weapons", "equipment", "consumables"] :
                        recipe = []
                        time.sleep(0.2)
                        try:
                            recipe_url = "https://www.dofus-touch.com" + cells[nom_index].find('a')['href']
                            recipe_response = requests.get(recipe_url, headers=headers)
                            recipe_soup = BeautifulSoup(recipe_response.text, "html.parser")
                            
                            # Here we find all elements with class 'ak-column' and 'ak-container'
                            recipe_elements = recipe_soup.find_all(class_='ak-column ak-container col-xs-12 col-sm-6')
                            
                            for element in recipe_elements:
                                # Find all <span> elements with class 'ak-linker'
                                span_elements = element.find_all('span', class_='ak-linker')
                                
                                # Loop through each <span> element and extract the text
                                for span in span_elements:
                                    text = span.text.strip()
                                    if text != '':
                                        recipe.append(text)
                        except:
                            continue 

                        object_info = {
                            "name": cells[nom_index].text.strip(),
                            "type": cells[type_index].text.strip(),
                            "level": cells[niveau_index].text.strip(),
                            "recipe": recipe
                        }
                    else :
                        object_info = {
                            "name": cells[nom_index].text.strip(),
                            "type": cells[type_index].text.strip(),
                            "level": cells[niveau_index].text.strip()
                        }
                    print(object_info)
                    object_list.append(object_info)
            return True  # Indique que la page a été analysée avec succès
    else:
        return False  # Indique qu'il y a eu une erreur lors de la requête

In [ ]:
base_url = "https://www.dofus-touch.com/fr/mmorpg/encyclopedia"
objects = {
    "weapons": [],
    "equipment": [],
    "consumables": [],
    "resources": []
}

for object_n, object_list in objects.items():
    print(f"Object : {object_n}")
    url = f"{base_url}/{object_n}"
    parse_object_page(url, object_n, object_list)
    page = 2

    while True:
        page_url = f"{url}?page={page}"
        success = parse_object_page(page_url, object_n, object_list)
        if not success:
            break
        page += 1
    print(f"{len(object_list)} {object_n} find")

In [ ]:
print("---------- test ----------")
print(f"weapons : {objects['weapons'][0]}")
print(f"equipment : {objects['equipment'][0]}")
print(f"consumables : {objects['consumables'][0]}")
print(f"resources : {objects['resources'][0]}")

In [ ]:
from neo4j import GraphDatabase

# Define your connection details
uri = "bolt://localhost:7687"
username = "your_username"
password = "your_password"

# Define your data
objects = {
    "weapons": ["sword", "bow", "axe"],
    "equipment": ["armor", "shield", "helmet"],
    "consumables": ["potion", "elixir", "food"],
    "resources": ["gold", "gems", "wood"]
}

# Define a function to create nodes for each type of object
def create_objects(tx):
    for obj_type, obj_list in objects.items():
        for obj in obj_list:
            tx.run(f"CREATE (:Object {{type: '{obj_type}', name: '{obj}'}})")

# Connect to the Neo4j database
driver = GraphDatabase.driver(uri, auth=(username, password))

# Execute the Cypher query to create nodes for objects
with driver.session() as session:
    session.write_transaction(create_objects)

# Close the driver when finished
driver.close()